# GridSearch

### IMPORTS

In [ ]:
import tensorrt # Only if using GPU's on Juno Cluster
import tensorflow as tf
import pandas as pd
import importlib
import IsoLearner
import visualization as vis
import model

importlib.reload(vis)
importlib.reload(IsoLearner)
importlib.reload(model)

import numpy as np

%reload_ext autoreload
%autoreload 2
print('done')

### Create GridSearch Object

In [ ]:
nodes = {
    'layer_1': [128, 256, 512],
    'layer_2': [128, 256, 512],
    'layer_3': [128, 256, 512],
    'layer_4': [128, 256, 512],
    'layer_5': [128, 256, 512],
    'layer_6': [128, 256, 512],
    'layer_7': [128, 256, 512]
}

activation_functions = ['relu', 'tanh']

learning_rate = 3e-05

dropout = [0.25, 0.5]

In [ ]:
GridSearch = model.model(nodes=nodes,
                         activation_functions=activation_functions, 
                         learning_rate=learning_rate, 
                         dropout=dropout,
                         lambda_val=0.001,)

## Build all Models

In [ ]:
models = GridSearch.build_models()

In [ ]:
# Save Models
for i in range(len(models)):
    models[i].save(f'./all-models/model-{i}.h5')

## Build IsoLearner Object

In [ ]:
Brain_Glucose_IsoLearner = IsoLearner.IsoLearner(absolute_data_path = "/juno/work/reznik/ilan/generated-data",
                                            relative_data_path = "brain-m0-no-log",
                                            morans_path = 'valid-metabs-brain-glucose.txt', 
                                            FML=False,
                                            tracer = 'glucose')

In [ ]:
# Make sure shapes are consistent 
for i in range(len(Brain_Glucose_IsoLearner.clean_ion_data)):
    print(Brain_Glucose_IsoLearner.clean_ion_data[i].shape, Brain_Glucose_IsoLearner.clean_iso_data[i].shape)

In [ ]:
# Training
Brain_Glucose_IsoLearner.cross_validation_training()

## Extract Data

In [ ]:
val_mae_ranked = {}
for i in range(len(Brain_Glucose_IsoLearner.all_models)):
    val_mae_ranked.update({i: Brain_Glucose_IsoLearner.all_models[i][3][1][-1]})

In [ ]:
val_mae_ranked

In [ ]:
val_mae_ranked = sorted(val_mae_ranked.items(), key=lambda x: x[1])
val_mae_ranked

In [ ]:
best_model = val_mae_ranked[0]
print(f'THE BEST MODEL IS MODEL: #{best_model[0]}, which has a testing MAE of {best_model[1]}')

In [ ]:
best_config = models[best_model[0]].get_config()

In [ ]:
nodes_layer_1 = best_config['layers'][1]['config']['units']
activation_layer_1 = best_config['layers'][1]['config']['activation']
nodes_layer_2 = best_config['layers'][3]['config']['units']
activation_layer_2 = best_config['layers'][3]['config']['activation']
nodes_layer_3 = best_config['layers'][5]['config']['units']
activation_layer_3 = best_config['layers'][5]['config']['activation']
dropout_1 = best_config['layers'][7]['config']['rate']
nodes_layer_4 = best_config['layers'][8]['config']['units']
activation_layer_4 = best_config['layers'][8]['config']['activation']
nodes_layer_5 = best_config['layers'][10]['config']['units']
activation_layer_5 = best_config['layers'][10]['config']['activation']
nodes_layer_6 = best_config['layers'][12]['config']['units']
activation_layer_6 = best_config['layers'][12]['config']['activation']
dropout_2 = best_config['layers'][14]['config']['rate']
nodes_layer_7 = best_config['layers'][15]['config']['units']
activation_layer_7 = best_config['layers'][15]['config']['activation']
nodes_layer_final = best_config['layers'][17]['config']['units']
activation_layer_final = best_config['layers'][17]['config']['activation']

In [ ]:
best_model_data = {
    'Layer 1': {
        '# Nodes': nodes_layer_1,
        'Activation': activation_layer_1,
    },
    'Layer 2': { 
        '# Nodes': nodes_layer_2,
        'Activation': activation_layer_2,
    },
    'Layer 3': {
        '# Nodes': nodes_layer_3,
        'Activation': activation_layer_3,
    },
    'Dropout 1': dropout_1,
    'Layer 4': { 
        '# Nodes': nodes_layer_4,
        'Activation': activation_layer_4,
    },
    'Layer 5': {
        '# Nodes': nodes_layer_5,
        'Activation': activation_layer_5,
    },
    'Layer 6': { 
        '# Nodes': nodes_layer_6,
        'Activation': activation_layer_6,
    },
    'Dropout 2': dropout_2,
    'Layer 7': {
        '# Nodes': nodes_layer_7,
        'Activation': activation_layer_7,
    },
    'Final Layer': { 
        '# Nodes': nodes_layer_final,
        'Activation': activation_layer_final,
    }

}

In [ ]:
best_model_data